In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow_hub as hub #contains a library of pre-trained models which includes the pre trained BERT model that we are going to fine tune
import tensorflow_text as text #this is a module which allows text to be converted into a format that is readable by the BERT model through pre-processing
import tensorflow as tenflow #the main tensorflow module used adding layers to the model 
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt
import seaborn as sn
from keras.models import load_model

def embedSentence(data):
    textProcess = bertText(data)
    return bertEncode(textProcess)['pooled_output']


simpleSpam = ["Dear Lucky Winner, Congratulations! You have been selected as one of our exclusive winners. To claim your prize, simply provide your contact details below. Don't miss out on this once-in-a-lifetime opportunity! Thank you for participating! Best regards, Prize Claim Department","Body: Dear [Recipient Name],As a token of appreciation for your continued support, we're delighted to offer you a complimentary gift! Simply click the link below to claim your free gift before supplies run out.[Link to Fake Gift Claim Page]Thank you for being a valued customer!Warm regards,Customer Loyalty Team","Body: Dear [Recipient Name],We wanted to remind you that your account subscription is due for renewal. To avoid any interruption in service, please update your payment information by clicking the link below.[Link to Fake Subscription Renewal Page]Thank you for your cooperation.Best regards,Account Management Team"]
descriptiveSpam = ["Body: Dear [Recipient Name],We have detected an unauthorized login attempt on your account originating from an unfamiliar location. To secure your account and prevent potential unauthorized access, please verify your identity by completing the security steps outlined in the email.Step 1: Click the link below to log in to your account securely.[Link to Fake Secure Login Page]Step 2: Once logged in, navigate to the 'Security Settings' section to review recent activity and update your account security preferences.Thank you for your prompt attention to this matter.Best regards,Account Security Team","Body: Dear [Recipient Name],You've been selected as one of our exclusive winners in our monthly prize draw! Your prize awaits – claim it now to unlock exciting rewards.Prize Details: [Description of Prize]To claim your prize, simply click the link below and follow the instructions on the prize claim page.[Link to Fake Prize Claim Page]Congratulations again on your win!Warm regards,Prize Management Team","Body: Dear [Recipient Name],As a token of appreciation for your continued support, we're offering a special promotion: receive a free gift with every purchase!Gift Details: [Description of Gift]To redeem your free gift, simply add items to your cart and proceed to checkout. Your free gift will be automatically added to your order.Shop now to claim your free gift: [Link to Fake Shopping Page]Thank you for choosing us as your preferred shopping destination!Sincerely,Customer Experience Team"]
detailedSpam = ["Body: Dear [Recipient Name],We are writing to inform you of a critical security update regarding your account. Recent activity suggests potential unauthorized access, and we're taking immediate steps to secure your account.Security Alert Details: [Description of Alert]To secure your account, please log in using the link below and follow the prompts to update your security settings.[Link to Fake Account Security Update Page]Thank you for your prompt attention to this matter.Sincerely,Security Team","Body: Dear [Recipient Name],We're excited to announce the launch of our latest features, and as a valued member of our community, you're invited to be among the first to experience them!New Feature Highlights: [Description of Features]To explore the latest enhancements to our platform, simply log in to your account using the link below.[Link to Fake Account Login Page]We can't wait for you to discover what's new!Best regards,Product Development Team","Body: Dear [Recipient Name],This is your last chance to take advantage of our exclusive offer! Don't miss out on your opportunity to save big on our premium products.Offer Details: [Description of Offer]Shop now before it's too late: [Link to Fake Shopping Page]Thank you for your continued support!Warm regards,Sales Team"]
simplePhish = ["Body: Dear Customer,We are currently updating our systems and require all customers to verify their accounts. To avoid any interruption in service, please click the link below and enter your account details.[Link to Fake Account Verification Page]Thank you for your cooperation.Best regards,Customer Support Team","Body: Dear Valued Customer,We have noticed unusual activity on your account. For your security, we have temporarily suspended your account. Please click the link below to verify your identity and reactivate your account.[Link to Fake Account Reactivation Page]Thank you for your prompt attention to this matter.Sincerely,Account Security Team","Body: Dear Customer,Congratulations! You have been selected to receive a special reward. To claim your reward, please click the link below and provide your contact information.[Link to Fake Reward Claim Page]Thank you for being a loyal customer.Warm regards,Promotions Team"]
descriptivePhish = ["Dear Rafferty, We have detected an unauthorized login attempt on your account originating from an unfamiliar location. To secure your account and prevent potential unauthorized access, please verify your identity by completing the security steps outlined in the email. Step 1: Click the link below to log in to your account securely. http://amaz0n.co.uk/login Step 2: Once logged in, navigate to the 'Security Settings' section to review recent activity and update your account security preferences. Thank you for your prompt attention to this matter. Best regards, Account Security Team","Body: Dear [Recipient Name],We regret to inform you that your account security has been compromised. Our system detected unauthorized access attempts originating from multiple locations.To secure your account and prevent further unauthorized access, please follow these steps:1. Change your account password immediately.2. Enable two-factor authentication for added security.3. Review recent account activity for any suspicious transactions.Thank you for your cooperation in protecting your account.Best regards,Security Team","Body: Dear [Recipient Name],We are conducting a routine account verification process to ensure the security of our systems and your account information. Failure to complete this process may result in temporary suspension of your account.To verify your account, please log in using the link below and follow the instructions on the verification page.[Link to Fake Account Verification Page]Thank you for your prompt attention to this matter.Best regards,Account Management Team"]
detailedPhish = ["Dear Rafferty, We're excited to announce the launch of our latest features, and as a valued member of our community, you're invited to be among the first to experience them! We have begun to offer a wide range of new top of the line products in a reserved section for our exclusive members, with specially curated products using our new AI tool that uses your data to recommend new items to you! To explore the latest enhancements to our platform, simply log in to your account using the link below. http://amaz0n.co.uk/login We can't wait for you to discover what's new! Best regards, Product Development Team","Body: Dear Customer,We're reaching out to inform you of an important update regarding your account. To ensure seamless service and access to new features, please log in to your account and review the latest updates.[Link to Fake Account Login Page]Thank you for being a valued customer.Warm regards,Customer Support Team","Body: Dear [Recipient Name],We've recently updated our security protocols, and your account requires immediate attention to ensure compliance and security. Please log in to your account using the link below to review and update your account information.[Link to Fake Account Update Page]Thank you for your cooperation in maintaining the security of our services.Warm regards,Customer Support Team"]
workEmail = ["Hi Rafferty, An Employee Appraisal has been submitted for you. Please perform the following steps to review and acknowledge this appraisal: Open your browser to our secured portal: https://ultraframe.co.uk/employee-appraisal Input your username, email and password. More info will be sent to your inbox to move forward. Locate the ‘Employee Acknowledgement’ section. Please acknowledge the appraisal by selecting one of the three options in the Employee Acknowledgement section. Select the ‘Save’ button. Please acknowledge your approval by August 01, 2024. Thank you, Josh Morton","We suspect a security breach happened earlier this week. In order to prevent further damage, we need everyone to change their password immediately. Please click here to do that: Change Password Please do this right away. Thanks! Sincerely, IT"]
manualEmail = ["Hi Adrian, After having a look, it seems to be because your original quote has no postcode. After putting the postcode in, the bars have been reinforced to account for the load values. This, as well as the outlet change, caused the price increase. Many thanks, Raff Ireland Technical Administrator Ultraframe(UK)Ltd Email: mailto:rafferty.ireland@ultraframe.co.uk ","Hi Mick, Just drawing this roof and it’s been booked on as a classic but has no discount. Josh has had a look in Logix and it seems like they may be a Wendland customer, but we aren’t sure. It looks like they’ve dealt with you in the past, would you be able to confirm the roof style? Many thanks, Raff Ireland Technical Administrator Ultraframe(UK)Ltd Email: mailto:rafferty.ireland@ultraframe.co.uk ","Hi Zoe, I have changed the cresting and finial for you. This may be me missing the obvious, but I can’t see an outlet on your attached drawing. Is it at the end of the box gutter? Please advise. Many thanks, Raff Ireland Technical Administrator Ultraframe(UK)Ltd Email: mailto:rafferty.ireland@ultraframe.co.uk ","Good Afternoon Keiron, Just chasing up on the below. Please could you quick the below link asap to get access to the new sign off! http://ultraframe/sign-off.co.uk Many thanks, Raff Ireland Technical Administrator Ultraframe(UK)Ltd Email: mailto:rafferty.ireland@ultraframe.co.uk"]
perfectEmail = ["Good Morning Jemma, I have noticed that you have a large width opening on this roof. Unfortunately, the roof support you have provided is inadequate and will require amending. We can send over a guide with all the steps you need to take, as well as a list of materials you will need. However, we have recently introduced a new webservice, allowing our customers to create and amend roofs themselves. If you would like to follow this path please follow the below link: http://Ultraframe/roof-system/customer-details.co.uk Please get in contact with me or the team if any further details are needed! Many thanks, Raff"]
enron = pd.read_csv('/kaggle/input/enron-email-dataset/emails.csv', encoding = "ISO-8859-1")
phishset = pd.read_csv('/kaggle/input/phishingemails/Phishing_Email.csv', encoding = "ISO-8859-1")
spam = pd.read_csv('/kaggle/input/email-spam-classification-dataset/combined_data.csv', encoding = "ISO-8859-1")
phishset.drop(columns=['Unnamed: 0'], inplace = True)
enron.drop(columns=['file'], inplace = True)
enron.insert(1, "Email Type", "Safe Email", True)
enron.rename(columns = {"message" : "Email Text"}, inplace = True)
spam = spam.reindex(columns = ['text', 'label'])
spam.rename(columns = {"text" : "Email Text"}, inplace = True)
spam.rename(columns = {"label" : "Email Type"}, inplace = True)
spamSafe = spam[spam['Email Type'] == 0]
spamFish = spam[spam['Email Type'] == 1]
spamSafe.drop(columns=['Email Type'], inplace = True)
spamFish.drop(columns=['Email Type'], inplace=True)
spamSafe.insert(1, "Email Type", "Safe Email", True)
spamFish.insert(1, "Email Type", "Phishing Email", True)
spamFishBalanced = spamFish.sample(spamSafe.shape[0])
balancedSpam = pd.concat([spamFishBalanced, spamSafe])
phishset = pd.concat([phishset, enron, balancedSpam])
phishsetSafe = phishset[phishset['Email Type'] == 'Safe Email']
phishsetFish = phishset[phishset['Email Type'] == 'Phishing Email']
phishsetSafeBalanced = phishsetSafe.sample(phishsetFish.shape[0])
balancedFish = pd.concat([phishsetSafeBalanced, phishsetFish])
balancedFish['Email Type'].value_counts()
balancedFish['phish'] = balancedFish['Email Type'].apply(lambda x:1 if x=='Phishing Email' else 0)
balancedFish.fillna(value = '', inplace=True)
balancedFish = balancedFish.sample(14656)
Training1, Testing1, Training2, Testing2 = train_test_split(balancedFish['Email Text'], balancedFish['phish'],test_size = 0.3, stratify = balancedFish['phish'])
Testing1, Validation1, Testing2, Validation2 = train_test_split(Testing1, Testing2, test_size = 0.33)
bertText = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bertEncode = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
Input = tenflow.keras.layers.Input(shape=(), dtype=tenflow.string, name='text')
processInput = bertText(Input)
Output = bertEncode(processInput)
dropout = tenflow.keras.layers.Dropout(0.1, name="dropout")(Output['pooled_output'])
dropout = tenflow.keras.layers.Dense(1, activation='sigmoid', name="output")(dropout)
finalModel = tenflow.keras.Model(inputs=Input,outputs = dropout)
modelMetrics = [tenflow.keras.metrics.BinaryAccuracy(name='accuracy'), tenflow.keras.metrics.Precision(name='precision'), tenflow.keras.metrics.Recall(name='recall')]
finalModel.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = modelMetrics)
finalModel.fit(Training1,Training2,epochs=4,validation_data= (Validation1, Validation2))
print("Simple Spam")
simpleS = finalModel.predict(simpleSpam)
print(simpleS)
print("Descriptive Spam")
descS = finalModel.predict(descriptiveSpam)
print(descS)
print("Detailed Spam")
detS = finalModel.predict(detailedSpam)
print(detS)
print("Simple Phish")
simP = finalModel.predict(simplePhish)
print(simP)
print("Descriptive Phish")
desP = finalModel.predict(descriptivePhish)
print(desP)
print("Detailed Phish")
detP = finalModel.predict(detailedPhish)
print(detP)
print("Work Emails")
workEmailPred = finalModel.predict(workEmail)
print(workEmailPred)
print("Manual Emails")
manualEmailPred = finalModel.predict(manualEmail)
print(manualEmailPred)
print("Perfect Email")
perfEmailPred = finalModel.predict(perfectEmail)
print(perfEmailPred)
#finalModel.save('/kaggle/working/finalModel.h5')
#finalModel.evaluate(Testing1, Testing2)
#finalPredicted = finalModel.predict(Training1)
#finalPredicted = finalPredicted.flatten()
#finalPredicted = np.where(finalPredicted > 0.5, 1, 0)
#finalPredicted
#confusionMatrix = confusion_matrix(Training2, finalPredicted)
#confusionMatrix
#sn.heatmap(confusionMatrix, annot = True, fmt='d')
#plt.xlabel('Predicted')
#plt.ylabel('Truth')
#print(classification_report(Training2, finalPredicted))










2024-08-15 13:23:19.616930: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 13:23:19.617055: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 13:23:19.746838: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_18/3967924763.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spamSafe.drop(columns=['Email Type'], inplace = True)
/tmp/ipykernel_18/3967924763.py:40: Settin

Epoch 1/4
321/321 [==============================] - 2454s 8s/step - loss: 0.4163 - accuracy: 0.8601 - precision: 0.8701 - recall: 0.8439 - val_loss: 0.2954 - val_accuracy: 0.9112 - val_precision: 0.9115 - val_recall: 0.9077
Epoch 2/4
321/321 [==============================] - 2431s 8s/step - loss: 0.2448 - accuracy: 0.9328 - precision: 0.9296 - recall: 0.9353 - val_loss: 0.2274 - val_accuracy: 0.9318 - val_precision: 0.9151 - val_recall: 0.9497
Epoch 3/4
321/321 [==============================] - 2433s 8s/step - loss: 0.1982 - accuracy: 0.9409 - precision: 0.9353 - recall: 0.9463 - val_loss: 0.1979 - val_accuracy: 0.9387 - val_precision: 0.9140 - val_recall: 0.9664
Epoch 4/4
321/321 [==============================] - 2444s 8s/step - loss: 0.1748 - accuracy: 0.9439 - precision: 0.9347 - recall: 0.9534 - val_loss: 0.1782 - val_accuracy: 0.9394 - val_precision: 0.9163 - val_recall: 0.9650
Simple Spam
1/1 [==============================] - 2s 2s/step
[[0.9396376 ]
 [0.95527935]
 [0.792962